In [2]:
import sys, os 
import gym 
import numpy as np
import gym_sudoku
import warnings 
warnings.catch_warnings()

catch_warnings()

In [45]:
sys.path.append("../")
from utils import get_solutions, check_solution

## Linear Programming Formulation of Sudoku

Reference: [Pulp Docs](https://www.coin-or.org/PuLP/CaseStudies/a_sudoku_problem.html)

In [3]:
env = gym.make('Sudoku-v0')
env.reset()
print(env.grid)

[[0 0 5 0 0 1 3 8 6]
 [1 0 7 8 0 5 0 9 2]
 [2 0 8 0 0 0 0 0 7]
 [5 0 0 0 0 0 1 6 0]
 [8 0 6 5 1 0 9 2 4]
 [4 0 0 0 0 0 8 0 5]
 [0 9 0 0 5 7 2 0 8]
 [7 0 0 1 0 2 0 5 9]
 [0 0 2 4 0 8 0 0 0]]


/home/james/anaconda3/envs/py10_sudoku_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (9, 9)
  logger.warn(
/home/james/anaconda3/envs/py10_sudoku_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/james/anaconda3/envs/py10_sudoku_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/james/anaconda3/

In [4]:
puzzle = env.base

In [5]:
puzzle

array([[0, 0, 5, 0, 0, 1, 3, 8, 6],
       [1, 0, 7, 8, 0, 5, 0, 9, 2],
       [2, 0, 8, 0, 0, 0, 0, 0, 7],
       [5, 0, 0, 0, 0, 0, 1, 6, 0],
       [8, 0, 6, 5, 1, 0, 9, 2, 4],
       [4, 0, 0, 0, 0, 0, 8, 0, 5],
       [0, 9, 0, 0, 5, 7, 2, 0, 8],
       [7, 0, 0, 1, 0, 2, 0, 5, 9],
       [0, 0, 2, 4, 0, 8, 0, 0, 0]])

## LP Formulation

In [6]:
seq = list(range(1,10))
vals = seq.copy()
rows = seq.copy()
cols = seq.copy()

# Approach
1. Just going to 'maximize' or 'minimize' zero since its a constraint satisfaction problem 
2. Create 729 decision varibles. Each is binary, telling whether choices 1 - 9 are present on each of the 81 cells. 

In [7]:
9**3

729

In [31]:
from pulp import (LpProblem, 
    LpVariable, 
    LpInteger, 
    LpMinimize, 
    lpSum,
    LpStatus,
    value)


In [9]:
boxes = [] 
for i in range(3):
    for j in range(3):
        boxes += [[(rows[3*i+k], cols[3*j+l]) for k in range(3) for l in range(3)]]

In [10]:
# each row contains the row,col pairs of each cell in a box
boxes

[[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)],
 [(1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6)],
 [(1, 7), (1, 8), (1, 9), (2, 7), (2, 8), (2, 9), (3, 7), (3, 8), (3, 9)],
 [(4, 1), (4, 2), (4, 3), (5, 1), (5, 2), (5, 3), (6, 1), (6, 2), (6, 3)],
 [(4, 4), (4, 5), (4, 6), (5, 4), (5, 5), (5, 6), (6, 4), (6, 5), (6, 6)],
 [(4, 7), (4, 8), (4, 9), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9)],
 [(7, 1), (7, 2), (7, 3), (8, 1), (8, 2), (8, 3), (9, 1), (9, 2), (9, 3)],
 [(7, 4), (7, 5), (7, 6), (8, 4), (8, 5), (8, 6), (9, 4), (9, 5), (9, 6)],
 [(7, 7), (7, 8), (7, 9), (8, 7), (8, 8), (8, 9), (9, 7), (9, 8), (9, 9)]]

In [11]:
prob = LpProblem("Sudoku", LpMinimize)

In [12]:
prob

Sudoku:
MINIMIZE
None
VARIABLES

In [13]:
choices = LpVariable.dicts("Choice", (vals,rows,cols),0,1,LpInteger)

In [14]:
choices

{1: {1: {1: Choice_1_1_1,
   2: Choice_1_1_2,
   3: Choice_1_1_3,
   4: Choice_1_1_4,
   5: Choice_1_1_5,
   6: Choice_1_1_6,
   7: Choice_1_1_7,
   8: Choice_1_1_8,
   9: Choice_1_1_9},
  2: {1: Choice_1_2_1,
   2: Choice_1_2_2,
   3: Choice_1_2_3,
   4: Choice_1_2_4,
   5: Choice_1_2_5,
   6: Choice_1_2_6,
   7: Choice_1_2_7,
   8: Choice_1_2_8,
   9: Choice_1_2_9},
  3: {1: Choice_1_3_1,
   2: Choice_1_3_2,
   3: Choice_1_3_3,
   4: Choice_1_3_4,
   5: Choice_1_3_5,
   6: Choice_1_3_6,
   7: Choice_1_3_7,
   8: Choice_1_3_8,
   9: Choice_1_3_9},
  4: {1: Choice_1_4_1,
   2: Choice_1_4_2,
   3: Choice_1_4_3,
   4: Choice_1_4_4,
   5: Choice_1_4_5,
   6: Choice_1_4_6,
   7: Choice_1_4_7,
   8: Choice_1_4_8,
   9: Choice_1_4_9},
  5: {1: Choice_1_5_1,
   2: Choice_1_5_2,
   3: Choice_1_5_3,
   4: Choice_1_5_4,
   5: Choice_1_5_5,
   6: Choice_1_5_6,
   7: Choice_1_5_7,
   8: Choice_1_5_8,
   9: Choice_1_5_9},
  6: {1: Choice_1_6_1,
   2: Choice_1_6_2,
   3: Choice_1_6_3,
   4: Choice_1

#### Objective

In [15]:
prob

Sudoku:
MINIMIZE
None
VARIABLES

In [16]:
prob += 0 

#### Constraint - All selected choices for a given cell must add up to one

In [17]:
for r in rows:
    for c in cols:
        prob += lpSum([choices[v][r][c] for v in vals]) == 1, ""

### Constraint - Each Number can only occur once in each row, column and box

In [18]:
for v in vals:
    for r in rows:
        prob += lpSum([choices[v][r][c] for c in cols]) == 1, ""
    for c in cols:
        prob += lpSum([choices[v][r][c] for r in rows]) == 1, ""
    for b in boxes:
        prob += lpSum([choices[v][r][c] for (r,c) in b]) == 1,""

In [19]:
prob

Sudoku:
MINIMIZE
0
SUBJECT TO
_C1: Choice_1_1_1 + Choice_2_1_1 + Choice_3_1_1 + Choice_4_1_1 + Choice_5_1_1
 + Choice_6_1_1 + Choice_7_1_1 + Choice_8_1_1 + Choice_9_1_1 = 1

_C2: Choice_1_1_2 + Choice_2_1_2 + Choice_3_1_2 + Choice_4_1_2 + Choice_5_1_2
 + Choice_6_1_2 + Choice_7_1_2 + Choice_8_1_2 + Choice_9_1_2 = 1

_C3: Choice_1_1_3 + Choice_2_1_3 + Choice_3_1_3 + Choice_4_1_3 + Choice_5_1_3
 + Choice_6_1_3 + Choice_7_1_3 + Choice_8_1_3 + Choice_9_1_3 = 1

_C4: Choice_1_1_4 + Choice_2_1_4 + Choice_3_1_4 + Choice_4_1_4 + Choice_5_1_4
 + Choice_6_1_4 + Choice_7_1_4 + Choice_8_1_4 + Choice_9_1_4 = 1

_C5: Choice_1_1_5 + Choice_2_1_5 + Choice_3_1_5 + Choice_4_1_5 + Choice_5_1_5
 + Choice_6_1_5 + Choice_7_1_5 + Choice_8_1_5 + Choice_9_1_5 = 1

_C6: Choice_1_1_6 + Choice_2_1_6 + Choice_3_1_6 + Choice_4_1_6 + Choice_5_1_6
 + Choice_6_1_6 + Choice_7_1_6 + Choice_8_1_6 + Choice_9_1_6 = 1

_C7: Choice_1_1_7 + Choice_2_1_7 + Choice_3_1_7 + Choice_4_1_7 + Choice_5_1_7
 + Choice_6_1_7 + Choice_7_1

#### Convert Starting Board to 'Constraints' - i.e can't change them

In [20]:
print(puzzle)

[[0 0 5 0 0 1 3 8 6]
 [1 0 7 8 0 5 0 9 2]
 [2 0 8 0 0 0 0 0 7]
 [5 0 0 0 0 0 1 6 0]
 [8 0 6 5 1 0 9 2 4]
 [4 0 0 0 0 0 8 0 5]
 [0 9 0 0 5 7 2 0 8]
 [7 0 0 1 0 2 0 5 9]
 [0 0 2 4 0 8 0 0 0]]


In [21]:
for r in rows:
    for c in cols:
        if puzzle[r-1,c-1] != 0:
            print(f"Setting row {r-1} col {c-1} to {puzzle[r-1,c-1]}")
            
            prob += choices[puzzle[r-1,c-1]][r][c] == 1,""

Setting row 0 col 2 to 5
Setting row 0 col 5 to 1
Setting row 0 col 6 to 3
Setting row 0 col 7 to 8
Setting row 0 col 8 to 6
Setting row 1 col 0 to 1
Setting row 1 col 2 to 7
Setting row 1 col 3 to 8
Setting row 1 col 5 to 5
Setting row 1 col 7 to 9
Setting row 1 col 8 to 2
Setting row 2 col 0 to 2
Setting row 2 col 2 to 8
Setting row 2 col 8 to 7
Setting row 3 col 0 to 5
Setting row 3 col 6 to 1
Setting row 3 col 7 to 6
Setting row 4 col 0 to 8
Setting row 4 col 2 to 6
Setting row 4 col 3 to 5
Setting row 4 col 4 to 1
Setting row 4 col 6 to 9
Setting row 4 col 7 to 2
Setting row 4 col 8 to 4
Setting row 5 col 0 to 4
Setting row 5 col 6 to 8
Setting row 5 col 8 to 5
Setting row 6 col 1 to 9
Setting row 6 col 4 to 5
Setting row 6 col 5 to 7
Setting row 6 col 6 to 2
Setting row 6 col 8 to 8
Setting row 7 col 0 to 7
Setting row 7 col 3 to 1
Setting row 7 col 5 to 2
Setting row 7 col 7 to 5
Setting row 7 col 8 to 9
Setting row 8 col 2 to 2
Setting row 8 col 3 to 4
Setting row 8 col 5 to 8


In [22]:
prob.writeLP("Sudoku.lp")

[Choice_1_1_1,
 Choice_1_1_2,
 Choice_1_1_3,
 Choice_1_1_4,
 Choice_1_1_5,
 Choice_1_1_6,
 Choice_1_1_7,
 Choice_1_1_8,
 Choice_1_1_9,
 Choice_1_2_1,
 Choice_1_2_2,
 Choice_1_2_3,
 Choice_1_2_4,
 Choice_1_2_5,
 Choice_1_2_6,
 Choice_1_2_7,
 Choice_1_2_8,
 Choice_1_2_9,
 Choice_1_3_1,
 Choice_1_3_2,
 Choice_1_3_3,
 Choice_1_3_4,
 Choice_1_3_5,
 Choice_1_3_6,
 Choice_1_3_7,
 Choice_1_3_8,
 Choice_1_3_9,
 Choice_1_4_1,
 Choice_1_4_2,
 Choice_1_4_3,
 Choice_1_4_4,
 Choice_1_4_5,
 Choice_1_4_6,
 Choice_1_4_7,
 Choice_1_4_8,
 Choice_1_4_9,
 Choice_1_5_1,
 Choice_1_5_2,
 Choice_1_5_3,
 Choice_1_5_4,
 Choice_1_5_5,
 Choice_1_5_6,
 Choice_1_5_7,
 Choice_1_5_8,
 Choice_1_5_9,
 Choice_1_6_1,
 Choice_1_6_2,
 Choice_1_6_3,
 Choice_1_6_4,
 Choice_1_6_5,
 Choice_1_6_6,
 Choice_1_6_7,
 Choice_1_6_8,
 Choice_1_6_9,
 Choice_1_7_1,
 Choice_1_7_2,
 Choice_1_7_3,
 Choice_1_7_4,
 Choice_1_7_5,
 Choice_1_7_6,
 Choice_1_7_7,
 Choice_1_7_8,
 Choice_1_7_9,
 Choice_1_8_1,
 Choice_1_8_2,
 Choice_1_8_3,
 Choice_1_

In [23]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/james/anaconda3/envs/py10_sudoku_env/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/4ff4eaa96e0140d08bb47d1dd9972b64-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/4ff4eaa96e0140d08bb47d1dd9972b64-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 369 COLUMNS
At line 4785 RHS
At line 5150 BOUNDS
At line 5881 ENDATA
Problem MODEL has 364 rows, 730 columns and 2956 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried

1

In [24]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [48]:
LpStatus[prob.status]

'Optimal'

In [25]:
solved_puzzle = puzzle.copy()

In [40]:
value(choices[9][1][1])

1.0

In [41]:

for v in vals:
    for r in rows:
        for c in cols:
            if value(choices[v][r][c]) == 1.0:
                print(f"Adding {v} at {r},{c}")
                solved_puzzle[r-1,c-1] = v

Adding 1 at 1,6
Adding 1 at 2,1
Adding 1 at 3,8
Adding 1 at 4,7
Adding 1 at 5,5
Adding 1 at 6,2
Adding 1 at 7,3
Adding 1 at 8,4
Adding 1 at 9,9
Adding 2 at 1,4
Adding 2 at 2,9
Adding 2 at 3,1
Adding 2 at 4,2
Adding 2 at 5,8
Adding 2 at 6,5
Adding 2 at 7,7
Adding 2 at 8,6
Adding 2 at 9,3
Adding 3 at 1,7
Adding 3 at 2,2
Adding 3 at 3,4
Adding 3 at 4,9
Adding 3 at 5,6
Adding 3 at 6,3
Adding 3 at 7,1
Adding 3 at 8,5
Adding 3 at 9,8
Adding 4 at 1,2
Adding 4 at 2,7
Adding 4 at 3,5
Adding 4 at 4,6
Adding 4 at 5,9
Adding 4 at 6,1
Adding 4 at 7,8
Adding 4 at 8,3
Adding 4 at 9,4
Adding 5 at 1,3
Adding 5 at 2,6
Adding 5 at 3,7
Adding 5 at 4,1
Adding 5 at 5,4
Adding 5 at 6,9
Adding 5 at 7,5
Adding 5 at 8,8
Adding 5 at 9,2
Adding 6 at 1,9
Adding 6 at 2,5
Adding 6 at 3,2
Adding 6 at 4,8
Adding 6 at 5,3
Adding 6 at 6,6
Adding 6 at 7,4
Adding 6 at 8,7
Adding 6 at 9,1
Adding 7 at 1,5
Adding 7 at 2,3
Adding 7 at 3,9
Adding 7 at 4,4
Adding 7 at 5,2
Adding 7 at 6,8
Adding 7 at 7,6
Adding 7 at 8,1
Adding 7

In [42]:
solved_puzzle

array([[9, 4, 5, 2, 7, 1, 3, 8, 6],
       [1, 3, 7, 8, 6, 5, 4, 9, 2],
       [2, 6, 8, 3, 4, 9, 5, 1, 7],
       [5, 2, 9, 7, 8, 4, 1, 6, 3],
       [8, 7, 6, 5, 1, 3, 9, 2, 4],
       [4, 1, 3, 9, 2, 6, 8, 7, 5],
       [3, 9, 1, 6, 5, 7, 2, 4, 8],
       [7, 8, 4, 1, 3, 2, 6, 5, 9],
       [6, 5, 2, 4, 9, 8, 7, 3, 1]])

In [46]:
check_solution(solved_puzzle)

'Solved'